In [14]:
from procedure import generate_train_predict

N = 25

sol = []

for i in range(N):
    s = generate_train_predict()
    sol.append(s)
    print(f'Process: {(i+1)/N*100:.0f}%')

Process: 0.00%
Process: 4.00%
Process: 8.00%
Process: 12.00%
Process: 16.00%
Process: 20.00%
Process: 24.00%
Process: 28.00%
Process: 32.00%
Process: 36.00%
Process: 40.00%
Process: 44.00%
Process: 48.00%
Process: 52.00%
Process: 56.00%
Process: 60.00%
Process: 64.00%
Process: 68.00%
Process: 72.00%
Process: 76.00%
Process: 80.00%
Process: 84.00%
Process: 88.00%
Process: 92.00%
Process: 96.00%


In [15]:
# Standard uncertainty
import numpy as np

E = np.array([sol[i]['E'] for i in range(N)])  # expectations
u = np.array([sol[i]['u'] for i in range(N)])  # uncertainties

print('Average uncertainty of the model with the current hyperparameters')
print(f'Absolute:\t{u.mean():.0f}kNm')
print(f'Relative:\t{u.mean()/E.mean()*100.:1f}%')

Average uncertainty of the model with the current hyperparameters
Absolute:	4042kNm
Relative:	6.405504%


In [16]:
# Realization-to-realization uncertainty

print('Realization-to-realization uncertainty for the expected load')
print(f'Absolute:\t{E.std():.0f}kNm')
print(f'Relative:\t{E.std()/E.mean()*100:.1f}%')

Realization-to-realization uncertainty for the expected load
Absolute:	205kNm
Relative:	0.3%


In [20]:
# Compare statistics of predicted and true data

E_pred = E
E_true = np.array([np.mean(sol[i]['y_true']) for i in range(N)])
E_dev = (E_pred-E_true)/E_true


std_pred = np.array([np.std(sol[i]['y_pred']) for i in range(N)])
std_true = np.array([np.std(sol[i]['y_true']) for i in range(N)])
std_dev = (std_pred-std_true)/std_true



print(f'Average predicted expectation:\t{E_pred.mean():.0f}kNm')
print(f'Average true expectation:\t{E_true.mean():.0f}kNm')
print(f'Average deviation:\t\t{E_dev.mean()*100:.0f}%\n')

print(f'Average predicted standard deviation:\t{std_pred.mean():.0f}kNm')
print(f'Average true standard deviation:\t{std_true.mean():.0f}kNm')
print(f'Average deviation:\t\t{std_dev.mean()*100:.0f}%')

Average predicted expectation:	63106kNm
Average true expectation:	63078kNm
Average deviation:		0%

Average predicted standard deviation:	6963kNm
Average true standard deviation:	8041kNm
Average deviation:		-13%
